In [3]:
! python --version

Python 3.10.12


In [4]:
! %%capture
! pip install --upgrade pip
! pip install timm
! pip install torch
! pip install pillow
! pip install ipywidgets
! pip uninstall numpy -y    # uninstall existing numpy (some components need pre 2.0)
! pip install "numpy<2.0"   # install the latest numpy 1.x version
! pip install opencv-python
! pip install scikit-learn
! pip install scipy
! pip install matplotlib

/bin/bash: line 1: fg: no job control
  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-no

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/MyDrive/Dev/bird-species-embedding-model/notebooks/
%pwd

/content/drive/MyDrive/Dev/bird-species-embedding-model/notebooks


'/content/drive/MyDrive/Dev/bird-species-embedding-model/notebooks'

In [23]:
import torch
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
import time

def fine_tune(train_dir, valid_dir, num_classes):
  start = time.time()
  model_no_fc_ready_to_fine_tune = timm.create_model("resnet50d.a1_in1k", pretrained=True, num_classes=num_classes)
  model = model_no_fc_ready_to_fine_tune

  # Prepare dataset
  data_config = resolve_data_config({}, model=model)
  transform = create_transform(**data_config)
  train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
  val_dataset = datasets.ImageFolder(root=valid_dir, transform=transform)
  train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=8, pin_memory=True)
  val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=8, pin_memory=True)

  # Fine-tune the model
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"Using device: {device}")
  model.to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.AdamW(model.parameters(), lr=1e-4)

  num_epochs = 10
  for epoch in range(num_epochs):
      model.train()
      running_loss = 0.0
      for inputs, labels in train_loader:
          inputs, labels = inputs.to(device), labels.to(device)
          optimizer.zero_grad()
          outputs = model(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()
          running_loss += loss.item()
      print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

      # Evaluate on validation set
      model.eval()
      correct = 0
      total = 0
      with torch.no_grad():
          for inputs, labels in val_loader:
              inputs, labels = inputs.to(device), labels.to(device)
              outputs = model(inputs)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()
      print(f'Validation Accuracy: {100 * correct / total}%')

  end = time.time()
  elapsed_time = end - start
  elapsed_time = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
  print(f"Elapsed time: {elapsed_time}")

  return model

# fine_tune("../kaggle_data/bird-species/train", "../kaggle_data/bird-species/valid", 525)


In [8]:
model

ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Co

In [9]:
# Remove the classification head, because we just need the output of the embedding model, and not the classification head
# model.reset_classifier(0)  # save model with the classification head, remove this later after loading

In [9]:
model

ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Co

In [10]:
# import numpy as np

# def extract_embeddings(dataloader):
#     model.eval()
#     embeddings = []
#     labels_list = []
#     with torch.no_grad():
#         for inputs, labels in dataloader:
#             inputs = inputs.to(device)
#             emb = model(inputs)
#             embeddings.append(emb.cpu().numpy())
#             labels_list.append(labels.numpy())
#     return np.concatenate(embeddings), np.concatenate(labels_list)

# # Extract embeddings for the training and validation datasets
# # train_embeddings, train_labels = extract_embeddings(train_loader)
# # val_embeddings, val_labels = extract_embeddings(val_loader)


In [11]:
# print(f"Train embedding Shape: {train_embeddings.shape}")
# print(f"Train embedding item shape: {train_embeddings[0].shape}")

# print(f"Train embedding slice: {train_embeddings[0][:10]}")

# print(f"Test embedding Shape: {val_embeddings.shape}")



In [12]:
import torch

# Assuming `model` is your fine-tuned model
model_path = 'resnet50d_fine_tune_525_bird_species_with_fc.pth'

# Save the model's state dictionary
torch.save(model.state_dict(), model_path)


In [49]:
import torch
import torchvision.models as models
import torch.nn as nn
import timm

# Define the model architecture (same as the one used for fine-tuning)
resnet_model = timm.create_model('resnet50d', pretrained=False)
resnet_model.reset_classifier(525) # 10 is the number of classes in the dataset

# Load the saved state dictionary
model_path = './resnet50d_fine_tune_525_bird_species_with_fc.pth'
resnet_model.load_state_dict(torch.load(model_path))

# Set the model to evaluation mode if you are using it for inference
resnet_model.eval()

# If you are planning to continue training, set the model to training mode
# model.train()


ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Co

In [15]:
resnet_model

ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Co

In [50]:
# Remove the classification head, because we just need the output of the embedding model, and not the classification head
resnet_model.reset_classifier(0)  # remove the classification head

In [51]:
resnet_model

ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Co

In [52]:
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image
from torchvision import transforms
import numpy as np

def cosine_similarity_(embedding1, embedding2):
  embedding1 = np.array(embedding1).reshape(1, -1)
  embedding2 = np.array(embedding2).reshape(1, -1)
  return cosine_similarity(embedding1, embedding2)[0][0]

def generate_embedding(image_path, resnet_model):
    # Load and preprocess the image
    input_image = Image.open(image_path)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_tensor = input_tensor.unsqueeze(0) # Add batch dimension

    # Move the input tensor to the same device as the model
    input_tensor = input_tensor.to(next(resnet_model.parameters()).device) # Move input to the same device as the model

    # Generate embeddings
    with torch.no_grad():
        embedding = resnet_model(input_tensor)

    embedding = embedding.cpu().numpy()
    return embedding.flatten()

# generate_embedding("../kaggle_data/bird-species/test/ABBOTTS BABBLER/1.jpg", resnet_model)

In [53]:
# similar/same birds
embedding1 = generate_embedding("../kaggle_data/bird-species/test/ABBOTTS BABBLER/4.jpg", resnet_model)
embedding2 = generate_embedding("../kaggle_data/bird-species/test/ABBOTTS BABBLER/5.jpg", resnet_model)

similarity = cosine_similarity_(embedding1, embedding2)
print(f"Cosine Similarity of is {similarity}\n")

Cosine Similarity of is 0.7555575370788574



In [54]:
# dissimilar/different birds
embedding1 = generate_embedding("../kaggle_data/bird-species/test/ABBOTTS BABBLER/4.jpg", resnet_model)
embedding2 = generate_embedding("../kaggle_data/bird-species/test/ABBOTTS BOOBY/1.jpg", resnet_model)

similarity = cosine_similarity_(embedding1, embedding2)
print(f"Cosine Similarity of is {similarity}\n")

Cosine Similarity of is 0.4179297983646393



In [55]:
# dissimilar/different birds
embedding1 = generate_embedding("../kaggle_data/bird-species/test/ABBOTTS BABBLER/4.jpg", resnet_model)
embedding2 = generate_embedding("../kaggle_data/bird-species/test/YELLOW HEADED BLACKBIRD/1.jpg", resnet_model)

similarity = cosine_similarity_(embedding1, embedding2)
print(f"Cosine Similarity of is {similarity}\n")

Cosine Similarity of is 0.437725305557251



### Comments about using pre-trained model to generate embeddings
Now that we have fine-tuned our embedding model against our bird species dataset, we can then use it to generate embeddings for our bird species images. This is done by passing the images through the model and extracting the embeddings from the last layer of the model. Because we have removed the classification head, our model now just spits out the embeddings of the image we pass in.

So now (compared to just using the pre-trained model) similar images will have a substantially higher similarity than a dissimilar image. This is because the model has been fine-tuned to recognise the specific features of the bird species in our dataset.

For example, in the similar case we get a score of `0.90834`, while the dissimilar image case gets a score of `0.31791`. This is a significant difference compared to using the pre-trained model, and shows that the model has learned to differentiate between the two images better.

## Can we fine tune against a smaller dataset?

Actually, if all we need are the embeddings, we can then fine tune against a smaller dataset (like 10 classes), instead of all the 525 classes. I have observed that the embeddings produced still represent the disparate classes.  

In [25]:
%pwd

'/content/drive/MyDrive/Dev/bird-species-embedding-model/notebooks'

In [26]:
model = fine_tune("../kaggle_data/bird-species-ten/train", "../kaggle_data/bird-species-ten/valid", 10)
model

Using device: cuda
Epoch [1/10], Loss: 2.2021080886616424
Validation Accuracy: 70.0%
Epoch [2/10], Loss: 1.8838643934212478
Validation Accuracy: 96.0%
Epoch [3/10], Loss: 1.2918024109859092
Validation Accuracy: 96.0%
Epoch [4/10], Loss: 0.6217620086436179
Validation Accuracy: 96.0%
Epoch [5/10], Loss: 0.32744380597974737
Validation Accuracy: 98.0%
Epoch [6/10], Loss: 0.1964155118547234
Validation Accuracy: 98.0%
Epoch [7/10], Loss: 0.13545329445133023
Validation Accuracy: 98.0%
Epoch [8/10], Loss: 0.10909626158136948
Validation Accuracy: 98.0%
Epoch [9/10], Loss: 0.07987480530259657
Validation Accuracy: 98.0%
Epoch [10/10], Loss: 0.07917185496611923
Validation Accuracy: 98.0%
Elapsed time: 00:03:57


ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Co

In [36]:
# Remove the classification head, because we just need the output of the embedding model, and not the classification head
resnet_model = model
resnet_model.reset_classifier(0)  # remove the classification head

In [37]:
resnet_model

ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Co

In [41]:
# similar/same birds
embedding1 = generate_embedding("../kaggle_data/bird-species-ten/test/ABBOTTS BABBLER/4.jpg", resnet_model)
embedding2 = generate_embedding("../kaggle_data/bird-species-ten/test/ABBOTTS BABBLER/5.jpg", resnet_model)

similarity = cosine_similarity_(embedding1, embedding2)
print(f"Cosine Similarity of is {similarity}\n")

Cosine Similarity of is 0.8841495513916016



In [44]:
# dissimilar/different birds
embedding1 = generate_embedding("../kaggle_data/bird-species-ten/test/ABBOTTS BABBLER/4.jpg", resnet_model)
embedding2 = generate_embedding("../kaggle_data/bird-species-ten/test/ABBOTTS BOOBY/1.jpg", resnet_model)

similarity = cosine_similarity_(embedding1, embedding2)
print(f"Cosine Similarity of is {similarity}\n")

Cosine Similarity of is 0.21176990866661072



In [46]:
# dissimilar/different birds
embedding1 = generate_embedding("../kaggle_data/bird-species-ten/test/ABBOTTS BABBLER/4.jpg", resnet_model)
embedding2 = generate_embedding("../kaggle_data/bird-species-ten/misc/YELLOW HEADED BLACKBIRD/1.jpg", resnet_model)

similarity = cosine_similarity_(embedding1, embedding2)
print(f"Cosine Similarity of is {similarity}\n")

Cosine Similarity of is 0.38036781549453735



In [48]:
import torch

# Assuming `model` is your fine-tuned model
model_path = 'resnet50d_fine_tune_10_bird_species_with_fc.pth'

# Save the model's state dictionary
torch.save(model.state_dict(), model_path)
